In [57]:
# load the data set
import pandas as pd

PATH="C:/Users/Owner/Desktop/projects\ML/chapter_2/ml_scans_changed.csv"
scans=pd.read_csv(PATH)

scans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   325         1200 non-null   float64
 1   335         1200 non-null   float64
 2   345         1200 non-null   float64
 3   355         1200 non-null   float64
 4   5           1200 non-null   float64
 5   15          1200 non-null   float64
 6   25          1200 non-null   float64
 7   35          1200 non-null   float64
 8   45          1200 non-null   float64
 9   direction   1200 non-null   float64
 10  correction  1200 non-null   float64
dtypes: float64(11)
memory usage: 103.2 KB


In [58]:
# separate train and test set
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(scans, test_size=0.2, random_state=42)
#train_set.info()
#test_set.info()

In [17]:
scans=train_set.copy()

In [59]:
## looking for correlation 
corr_matrix=scans.corr()
corr_matrix['correction'].sort_values(ascending=False)

correction    1.000000
345           0.260933
325           0.250161
335           0.206988
355           0.067320
5            -0.061917
15           -0.156459
25           -0.159918
35           -0.279820
45           -0.295252
direction    -0.438641
Name: correction, dtype: float64

In [60]:
############## prepare the data for ML(separate labels from predictors)
scans_labels = scans["correction"].copy() # labels
scans = scans.drop("correction", axis=1) # predictors(witout lables)
scans.iloc[:5]
scans_labels.iloc[:5]

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: correction, dtype: float64

In [26]:
#scans.info()

In [61]:
"""Training and Evaluating on the Training Set"""
# linear Regression
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(scans, scans_labels) # prepared data(parameters) and labels

LinearRegression()

In [62]:
# Test linear model
some_data = scans.iloc[40:45]
some_labels = scans_labels.iloc[40:45]


print("Predictions:", lin_reg.predict(some_data))
print("Labels:", list(some_labels))


Predictions: [-29.9563865186 -28.9972949083 -22.2808283394 -29.8796591898
  34.7822971803]
Labels: [-95.0, -80.0, -45.0, -85.0, 73.0]


In [63]:
"""DecisionTreeRegressor. This is a powerful model, capable of finding
complex nonlinear relationships in the data"""
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(scans, scans_labels)

DecisionTreeRegressor()

In [64]:
f_i=600
l_i=630
some_data = scans.iloc[f_i:l_i]
some_labels = scans_labels.iloc[f_i:l_i]

print("Predictions:", tree_reg.predict(some_data))
print("Labels:", list(some_labels))

Predictions: [-160. -165. -170. -175. -180.   40.   35.   30.   25.   20.   15.   10.
    5.    0.   -5.  -10.  -15.  -20.  -25.  -30.  -35.  -40.  -45.   70.
   65.   60.   55.   50.   45.   40.]
Labels: [-160.0, -165.0, -170.0, -175.0, -180.0, 40.0, 35.0, 30.0, 25.0, 20.0, 15.0, 10.0, 5.0, 0.0, -5.0, -10.0, -15.0, -20.0, -25.0, -30.0, -35.0, -40.0, -45.0, 70.0, 65.0, 60.0, 55.0, 50.0, 45.0, 40.0]


In [65]:
from sklearn.model_selection import cross_val_score
import numpy as np
np.set_printoptions(precision=10, suppress=True)

scores = cross_val_score(tree_reg, scans, scans_labels,
scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
print(tree_rmse_scores)

[81.987041659  80.956778592  74.3429552816 67.7961404998 65.532689044
 84.7748488645 35.3376569682  5.3424401416 15.518805796  21.0851448497]


In [67]:
''' Evaluate Your System on the Test Set '''
from sklearn.metrics import mean_squared_error
y_test = test_set["correction"].copy()
X_test = test_set.drop("correction", axis=1)

final_predictions = tree_reg.predict(X_test)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print(final_rmse) # => evaluates to 47,730.2

0.0


In [76]:
f_i=0
l_i=1
some_dat =X_test.iloc[f_i:l_i]
print(some_dat)
some_label = y_test.iloc[f_i:l_i]

print("Predictions:", tree_reg.predict(some_dat))
print("Labels:", list(some_label))


        325    335    345    355      5     15     25     35     45  direction
1178  179.0  624.0  386.0  340.0  311.0  227.0  162.0  159.0  162.0       46.0
Predictions: [0.]
Labels: [0.0]
